<a href="https://colab.research.google.com/github/tuhinmallick/AI-for-Fashion/blob/main/Fast_Fine_tuning_and_DPO_Training_for_Google_Gemma_with_Unsloth_(Zephyr_Recipe).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook shows how to fine-tune and align Google's Gemma with distilled supervised fine-tuning and distilled DPO, following the recipe proposed by Hugging Face for Zephyr Gemma but using unsloth for the faster and memory-efficient training.

The notebook has two parts: supervised fine-tuning (SFT) and DPO training.

unsloth exists in different versions optimized for different GPU/hardware configuration. Run the following cell to install all the packages for your configuration. To fully benefit from Unsloth's optimizing, an Ampere GPU, or more recent (NVIDIA RTX 30xx/40xx, A100, H100, etc), is recommended.

In [ ]:
import torch

major_version, minor_version = torch.cuda.get_device_capability()
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
  !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
  !pip install --no-deps xformers trl peft accelerate bitsandbytes

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-zia2kajd/unsloth_e892ec229ab24286b38e9c2640016035
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-zia2kajd/unsloth_e892ec229ab24286b38e9c2640016035
  Resolved https://github.com/unslothai/unsloth.git to commit 1e61cdbcb2a6c0c399d9e3e58a157ee1144ebf69
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.5 MB/s eta 0:00:00
  Created wheel for uns

Import the necessary libraries:

In [ ]:
from unsloth import FastLanguageModel
from trl import SFTTrainer
from transformers import TrainingArguments, AutoTokenizer
from datasets import load_dataset
import torch

max_seq_length = 2048
dtype = None
load_in_4bit = True

/usr/local/lib/python3.10/dist-packages/unsloth/__init__.py:71: UserWarning: Unsloth: Running `ldconfig /usr/lib64-nvidia` to link CUDA.
  warnings.warn(


Load the model with unsloth and the tokenizer used by Zephyr Gemma:

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-7b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

tokenizer = AutoTokenizer.from_pretrained("philschmid/gemma-tokenizer-chatml", use_fast=True)

config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Gemma patching release 2024.3
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.1.0+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.22.post7. FA = True.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


/usr/local/lib/python3.10/dist-packages/transformers/quantizers/auto.py:155: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


model.safetensors:   0%|          | 0.00/5.57G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/402 [00:00<?, ?B/s]

Initializing and mounting the LoRA adapter that will be fine-tuned:

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = True,
    random_state = 3407,
)

Unsloth 2024.3 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


Load the instruction dataset used for SFT:

In [ ]:
dataset = load_dataset("HuggingFaceH4/deita-10k-v0-sft", split=['train_sft','test_sft'])

Generating train_sft split:   0%|          | 0/9500 [00:00<?, ? examples/s]

Generating test_sft split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating train_gen split:   0%|          | 0/9500 [00:00<?, ? examples/s]

Generating test_gen split:   0%|          | 0/500 [00:00<?, ? examples/s]

Set up the training hyperparameters and the SFTTrainer's configuration:

In [ ]:
training_args = TrainingArguments(
        do_eval=True,
        evaluation_strategy = "steps",
        eval_steps = 100,
        save_strategy = "epoch",
        per_device_train_batch_size = 4, #Zephyr
        gradient_accumulation_steps = 4, #Zephyr
        per_device_eval_batch_size = 4,
        warmup_ratio = 0.1, #Zephyr
        num_train_epochs = 3, #Zephyr
        learning_rate = 2.0e-05, #Zephyr
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 100,
        optim = "adamw_8bit",
        lr_scheduler_type = "cosine", #Zephyr
        seed = 3407,
        output_dir = "./drive/MyDrive/gemma7b_SFT/",
)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset[0],
    eval_dataset = dataset[1],
    max_seq_length = max_seq_length,
    dataset_kwargs={
        "add_special_tokens": False, # We template with special tokens
        "append_concat_token": False, # No need to add additional separator token
    },
    args = training_args
)

Map:   0%|          | 0/9500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:294: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Start training:

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 9,500 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 4 | Gradient Accumulation steps = 4
\        /    Total batch size = 16 | Total steps = 1,779
 "-____-"     Number of trainable parameters = 50,003,968


Step,Training Loss,Validation Loss
100,1.880300,1.116232
200,1.141900,1.038970
300,1.048900,1.009894
400,1.020400,1.001383
500,0.973500,0.992200
600,0.979900,0.984375
700,0.959300,0.981631
800,0.949500,0.978139
900,0.952400,0.975801
1000,0.931000,0.972941


Step,Training Loss,Validation Loss
100,1.880300,1.116232
200,1.141900,1.038970
300,1.048900,1.009894
400,1.020400,1.001383
500,0.973500,0.992200
600,0.979900,0.984375
700,0.959300,0.981631
800,0.949500,0.978139
900,0.952400,0.975801
1000,0.931000,0.972941


# DPO Training



Import all the necessary, load the SFT model (obtained by the previous cell), and the same tokenizer:

In [ ]:
from unsloth import FastLanguageModel
from trl import DPOTrainer
from peft import PeftModel
from transformers import TrainingArguments, AutoTokenizer
from datasets import load_dataset
import torch

max_seq_length = 1024 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "./gemma7b_SFT/checkpoint-1779/",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

tokenizer = AutoTokenizer.from_pretrained("philschmid/gemma-tokenizer-chatml", use_fast=True)

config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Gemma patching release 2024.3
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.0+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.24. FA = True.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


/usr/local/lib/python3.10/dist-packages/transformers/quantizers/auto.py:155: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


model.safetensors:   0%|          | 0.00/5.57G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

Unsloth 2024.3 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


tokenizer_config.json:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/402 [00:00<?, ?B/s]

Load and format the data to be used by the DPOTrainer:

In [ ]:
dataset = load_dataset("argilla/dpo-mix-7k")
column_names = list(dataset["train"].features)
def apply_dpo_template(example):
  if all(k in example.keys() for k in ("chosen", "rejected")):
    # For DPO, the inputs are triples of (prompt, chosen, rejected), where `chosen` and `rejected` are the final turn of a dialogue
    # We therefore need to extract the N-1 turns to form the prompt
    prompt_messages = example["chosen"][:-1]


    # Now we extract the final turn to define chosen/rejected responses
    chosen_messages = example["chosen"][-1:]
    rejected_messages = example["rejected"][-1:]
    example["text_chosen"] = tokenizer.apply_chat_template(chosen_messages, tokenize=False)
    example["text_rejected"] = tokenizer.apply_chat_template(rejected_messages, tokenize=False)
    example["text_prompt"] = tokenizer.apply_chat_template(prompt_messages, tokenize=False)
  return example

dataset = dataset.map(apply_dpo_template,remove_columns=column_names,
          desc="Formatting comparisons with prompt template",)
for split in ["train", "test"]:
    dataset[split] = dataset[split].rename_columns(
        {"text_prompt": "prompt", "text_chosen": "chosen", "text_rejected": "rejected"}
    )

Generating train split:   0%|          | 0/6750 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/750 [00:00<?, ? examples/s]

Formatting comparisons with prompt template:   0%|          | 0/6750 [00:00<?, ? examples/s]

Formatting comparisons with prompt template:   0%|          | 0/750 [00:00<?, ? examples/s]

In [ ]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['chosen', 'rejected', 'prompt'],
        num_rows: 6750
    })
    test: Dataset({
        features: ['chosen', 'rejected', 'prompt'],
        num_rows: 750
    })
})


Set the hyperparameters, patch the DPOTrainer with unsloth, and set up the DPOTrainer configuration:

In [ ]:
training_args = TrainingArguments(
        do_eval=True,
        evaluation_strategy = "steps",
        eval_steps = 100,
        save_strategy = "epoch",
        per_device_train_batch_size = 1, #Zephyr
        gradient_accumulation_steps = 16, #Zephyr
        per_device_eval_batch_size = 1,
        warmup_ratio = 0.1, #Zephyr
        num_train_epochs = 2, #Zephyr
        learning_rate = 5.0e-07, #Zephyr
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 100,
        optim = "paged_adamw_8bit",
        lr_scheduler_type = "cosine", #Zephyr
        seed = 3407,
        output_dir = "./gemma7b_DPO/",
)


from unsloth import PatchDPOTrainer
PatchDPOTrainer()

trainer = DPOTrainer(
    model,
    ref_model=None,
    args=training_args,
    beta=0.05, #Zephyr
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    tokenizer=tokenizer
)



/usr/local/lib/python3.10/dist-packages/trl/trainer/dpo_trainer.py:296: UserWarning: `max_length` is not set in the DPOTrainer's init it will default to `512` by default, but you should do it yourself in the future.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/dpo_trainer.py:303: UserWarning: `max_prompt_length` is not set in the DPOTrainer's init it will default to `128` by default, but you should do it yourself in the future.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/dpo_trainer.py:328: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(


Map:   0%|          | 0/6750 [00:00<?, ? examples/s]

Map:   0%|          | 0/750 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Start DPO training:

In [ ]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 6,750 | Num Epochs = 2
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 16
\        /    Total batch size = 16 | Total steps = 842
 "-____-"     Number of trainable parameters = 50,003,968
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss,Validation Loss,rewards / chosen,rewards / rejected,rewards / accuracies,rewards / margins,logps / rejected,logps / chosen,logits / rejected,logits / chosen
100,3.175500,1.798390,10.323790,9.904058,0.577333,0.419733,-2267.950928,-2177.997070,45.572098,27.104448
200,1.633400,1.248681,2.748679,2.168667,0.616000,0.580013,-2422.658447,-2329.499268,39.244755,20.696583
300,1.370700,1.158425,3.462418,2.886935,0.622667,0.575483,-2408.293457,-2315.224365,38.623173,20.105232
400,1.393700,1.103643,3.760677,3.200983,0.625333,0.559694,-2402.012695,-2309.259277,37.589203,19.092251
500,1.211400,1.053873,3.057409,2.453395,0.633333,0.604014,-2416.964355,-2323.324707,36.749805,18.288084
600,1.159100,1.039545,2.291616,1.610981,0.650667,0.680634,-2433.812500,-2338.640381,36.523201,18.097391


Checkpoint destination directory ./drive/MyDrive/gemma7b_DPO/checkpoint-421 already exists and is non-empty. Saving will proceed but saved results may be invalid.


Step,Training Loss,Validation Loss,rewards / chosen,rewards / rejected,rewards / accuracies,rewards / margins,logps / rejected,logps / chosen,logits / rejected,logits / chosen
100,3.175500,1.798390,10.323790,9.904058,0.577333,0.419733,-2267.950928,-2177.997070,45.572098,27.104448
200,1.633400,1.248681,2.748679,2.168667,0.616000,0.580013,-2422.658447,-2329.499268,39.244755,20.696583
300,1.370700,1.158425,3.462418,2.886935,0.622667,0.575483,-2408.293457,-2315.224365,38.623173,20.105232
400,1.393700,1.103643,3.760677,3.200983,0.625333,0.559694,-2402.012695,-2309.259277,37.589203,19.092251
500,1.211400,1.053873,3.057409,2.453395,0.633333,0.604014,-2416.964355,-2323.324707,36.749805,18.288084
600,1.159100,1.039545,2.291616,1.610981,0.650667,0.680634,-2433.812500,-2338.640381,36.523201,18.097391
700,1.170300,1.042234,2.166914,1.493485,0.650667,0.673429,-2436.162354,-2341.134277,36.417931,17.998878
800,1.175300,1.032524,2.345064,1.662173,0.642667,0.682891,-2432.788574,-2337.571777,36.458649,18.042673


TrainOutput(global_step=842, training_loss=1.5197757893107, metrics={'train_runtime': 8729.2345, 'train_samples_per_second': 1.547, 'train_steps_per_second': 0.096, 'total_flos': 0.0, 'train_loss': 1.5197757893107, 'epoch': 2.0})